In [1]:
import subprocess

import pandas as pd
import numpy as np
import openpyxl

In [2]:
TARGET_COLUMNS = [
    'hospital_id',
    #'row_id',
    'line_type',
    'description',
    'rev_code',
    'local_code',
    'code',
    'ms_drg',
    'apr_drg',
    'eapg',
    'hcpcs_cpt',
    'modifiers',
    'alt_hcpcs_cpt',
    'thru',
    'apc',
    'icd',
    'ndc',
    'drug_hcpcs_multiplier',
    'drug_quantity',
    'drug_unit_of_measurement',
    'drug_type_of_measurement',
    'billing_class',
    'setting',
    'payer_category',
    'payer_name',
    'plan_name',
    'standard_charge',
    'standard_charge_percent',
    'contracting_method',
    'additional_generic_notes',
    'additional_payer_specific_notes'
]

In [3]:
# https://www.piedmont.org/patient-tools/hospital-price-transparency
#ccn = "110083"
#url = "https://www.piedmont.org/media/file/580566213_piedmont-atlanta-hospital_standardcharges.xls"

#ccn = "110074"
#url = "https://www.piedmont.org/media/file/582179986_piedmont-athens-hospital_standardcharges.xls"

ccn = "110064"
url = "https://www.piedmont.org/media/file/581685139_piedmont-columbus-regional-midtown-hospital_standardcharges.xls"

filename = url.split("/")[-1].replace("%20", " ")
ein = filename.split("_")[0]
ein = ein[:2] + "-" + ein[2:]
ein

'58-1685139'

In [4]:
subprocess.run(["wget", "--no-clobber", url, "-O", filename])

--2023-05-20 14:09:14--  https://www.piedmont.org/media/file/581685139_piedmont-columbus-regional-midtown-hospital_standardcharges.xls
Resolving www.piedmont.org (www.piedmont.org)... 104.239.187.127
Connecting to www.piedmont.org (www.piedmont.org)|104.239.187.127|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13480448 (13M) [application/vnd.ms-excel]
Saving to: ‘581685139_piedmont-columbus-regional-midtown-hospital_standardcharges.xls’

     0K .......... .......... .......... .......... ..........  0%  372K 35s
    50K .......... .......... .......... .......... ..........  0%  669K 27s
   100K .......... .......... .......... .......... ..........  1% 59.8M 18s
   150K .......... .......... .......... .......... ..........  1%  741K 18s
   200K .......... .......... .......... .......... ..........  1%  799K 18s
   250K .......... .......... .......... .......... ..........  2% 8.32M 15s
   300K .......... .......... .......... .......... ..........  2%  

CompletedProcess(args=['wget', '--no-clobber', 'https://www.piedmont.org/media/file/581685139_piedmont-columbus-regional-midtown-hospital_standardcharges.xls', '-O', '581685139_piedmont-columbus-regional-midtown-hospital_standardcharges.xls'], returncode=0)

In [5]:
df_in = pd.read_excel(filename, skiprows=[1,2,3])
df_in

,Piedmont Columbus Regional Midtown Hospital
0,"Procedure,Description,HCPCS Code,Rev Code,MSDR..."
1,"18600179,K-WIRE SMOOTH 9 .035,C1713,278,N/A,N/..."
2,"18600415,WIRE FIXATION 12.7CM .028IN KIRSCHNER..."
3,"18602712,WIRE FIXATION 15.2CM .045IN KIRSCHNER..."
4,"18602720,WIRE FIXATION 15.2CM .078IN KIRSCHNER..."
...,...
56359,"WA22557C,ELECTRD ELECTROSURGICAL 30D BUTTON HI..."
56360,"X-4016,PLEDG CARDIOVASCULAR 10X7MM PTFE PRECUT..."
56361,"ZA90030155,LENS INTRAOCULAR +15.5 DIOPTER L13M..."
56362,"ZMA00U0185,LENS IOL 5 D +18.5 DIOPTER MODIFY C..."


In [6]:
# HACK!!!
d = df_in.iloc[0].to_dict()
k = list(d.keys())[0]
columns = d[k].split(',')
columns

['Procedure',
 'Description',
 'HCPCS Code',
 'Rev Code',
 'MSDRG',
 'NDC',
 'Charges',
 'Aetna HMO/POS/OA',
 'Ambetter',
 'Managed Medicaid - Amerigroup',
 'Blue Cross HMO',
 'Blue Cross PPO',
 'Managed Medicaid - Caresource',
 'Cigna HMO/POS/OA',
 'Managed Medicare - Cigna Healthspring',
 'Managed Medicare - Clover Health',
 'Cigna Local+',
 'Managed Medicare - Georgia Health Advantage',
 'Humana HMO',
 'Managed Medicare - Humana',
 'Kaiser',
 'Managed Medicare - Kaiser',
 'Multiplan',
 'Managed Medicaid - PeachState',
 'Managed Medicare - Pruitt Health',
 'Managed Medicare - Sonder',
 'PHCS',
 'Managed Medicare - Wellcare',
 'United Healthcare',
 'Self-Pay',
 'Min Fee',
 'Max Fee']

In [7]:
df_in = df_in.iloc[1:] 
records = df_in[k].to_list()
records = list(map(lambda r: r.split(","), records))

# Sometimes description is comma-separated, which shifts the indices by one.
# This little trick fixes that problem.
def fix_record_list(r):
    while len(r) > len(columns):
        r[1] = r[1] + "," + r[2]
        del r[2]
    
    return r

records = list(map(lambda r: fix_record_list(r), records))

In [8]:
records = list(map(lambda r: dict(zip(columns, r)), records))
df_in = pd.DataFrame(records)
df_in = df_in.replace("N/A", np.NaN) # https://stackoverflow.com/a/34794112
df_in

,Procedure,Description,HCPCS Code,Rev Code,MSDRG,NDC,Charges,Aetna HMO/POS/OA,Ambetter,Managed Medicaid - Amerigroup,...,Multiplan,Managed Medicaid - PeachState,Managed Medicare - Pruitt Health,Managed Medicare - Sonder,PHCS,Managed Medicare - Wellcare,United Healthcare,Self-Pay,Min Fee,Max Fee
0,18600179,K-WIRE SMOOTH 9 .035,C1713,278,NaN,NaN,74.88,62.15,0,19.72,...,50.92,19.72,0,0,50.92,0,0,22.46,0,62.15
1,18600415,WIRE FIXATION 12.7CM .028IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,34.77,28.86,0,9.16,...,23.64,9.16,0,0,23.64,0,0,10.43,0,28.86
2,18602712,WIRE FIXATION 15.2CM .045IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,34.77,28.86,0,9.16,...,23.64,9.16,0,0,23.64,0,0,10.43,0,28.86
3,18602720,WIRE FIXATION 15.2CM .078IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,40.77,33.84,0,10.74,...,27.72,10.74,0,0,27.72,0,0,12.23,0,33.84
4,18604400,WIRE FIXATION TROCAR POINT STYLE BOTH END 15.2...,C1713,278,NaN,NaN,47.76,39.64,0,12.58,...,32.47,12.58,0,0,32.47,0,0,14.33,0,39.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56358,WA22557C,ELECTRD ELECTROSURGICAL 30D BUTTON HIGH FREQUENCY,NaN,272,NaN,NaN,2264.62,0,0,0,...,0,0,0,0,0,0,0,679.38,679.38,1585.24
56359,X-4016,PLEDG CARDIOVASCULAR 10X7MM PTFE PRECUT STERILE,C1768,278,NaN,NaN,77.25,0,0,0,...,0,0,0,0,0,0,0,23.17,23.17,54.08
56360,ZA90030155,LENS INTRAOCULAR +15.5 DIOPTER L13MM DIA6MM RE...,V2632,276,NaN,NaN,754,0,0,0,...,0,0,0,0,0,0,0,226.2,226.2,527.8
56361,ZMA00U0185,LENS IOL 5 D +18.5 DIOPTER MODIFY C BICONVEX 1...,V2632,276,NaN,NaN,3723.2,0,0,0,...,0,0,0,0,0,0,0,1116.96,1116.96,2606.24


In [9]:
df_mid = pd.DataFrame(df_in)

df_mid = df_mid.rename(columns={
    'Procedure': 'local_code',
    'Description': 'description',
    'HCPCS Code': 'hcpcs_cpt',
    'MSDRG': 'ms_drg',
    'Rev Code': 'rev_code',
    'NDC': 'ndc'
})

df_mid

,local_code,description,hcpcs_cpt,rev_code,ms_drg,ndc,Charges,Aetna HMO/POS/OA,Ambetter,Managed Medicaid - Amerigroup,...,Multiplan,Managed Medicaid - PeachState,Managed Medicare - Pruitt Health,Managed Medicare - Sonder,PHCS,Managed Medicare - Wellcare,United Healthcare,Self-Pay,Min Fee,Max Fee
0,18600179,K-WIRE SMOOTH 9 .035,C1713,278,NaN,NaN,74.88,62.15,0,19.72,...,50.92,19.72,0,0,50.92,0,0,22.46,0,62.15
1,18600415,WIRE FIXATION 12.7CM .028IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,34.77,28.86,0,9.16,...,23.64,9.16,0,0,23.64,0,0,10.43,0,28.86
2,18602712,WIRE FIXATION 15.2CM .045IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,34.77,28.86,0,9.16,...,23.64,9.16,0,0,23.64,0,0,10.43,0,28.86
3,18602720,WIRE FIXATION 15.2CM .078IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,40.77,33.84,0,10.74,...,27.72,10.74,0,0,27.72,0,0,12.23,0,33.84
4,18604400,WIRE FIXATION TROCAR POINT STYLE BOTH END 15.2...,C1713,278,NaN,NaN,47.76,39.64,0,12.58,...,32.47,12.58,0,0,32.47,0,0,14.33,0,39.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56358,WA22557C,ELECTRD ELECTROSURGICAL 30D BUTTON HIGH FREQUENCY,NaN,272,NaN,NaN,2264.62,0,0,0,...,0,0,0,0,0,0,0,679.38,679.38,1585.24
56359,X-4016,PLEDG CARDIOVASCULAR 10X7MM PTFE PRECUT STERILE,C1768,278,NaN,NaN,77.25,0,0,0,...,0,0,0,0,0,0,0,23.17,23.17,54.08
56360,ZA90030155,LENS INTRAOCULAR +15.5 DIOPTER L13MM DIA6MM RE...,V2632,276,NaN,NaN,754,0,0,0,...,0,0,0,0,0,0,0,226.2,226.2,527.8
56361,ZMA00U0185,LENS IOL 5 D +18.5 DIOPTER MODIFY C BICONVEX 1...,V2632,276,NaN,NaN,3723.2,0,0,0,...,0,0,0,0,0,0,0,1116.96,1116.96,2606.24


In [10]:
money_columns = df_mid.columns[6:]
remaining_columns = df_mid.columns[:6]
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name', value_name='standard_charge')
df_mid

,local_code,description,hcpcs_cpt,rev_code,ms_drg,ndc,payer_name,standard_charge
0,18600179,K-WIRE SMOOTH 9 .035,C1713,278,NaN,NaN,Charges,74.88
1,18600415,WIRE FIXATION 12.7CM .028IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,Charges,34.77
2,18602712,WIRE FIXATION 15.2CM .045IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,Charges,34.77
3,18602720,WIRE FIXATION 15.2CM .078IN KIRSCHNER STAINLES...,C1713,278,NaN,NaN,Charges,40.77
4,18604400,WIRE FIXATION TROCAR POINT STYLE BOTH END 15.2...,C1713,278,NaN,NaN,Charges,47.76
...,...,...,...,...,...,...,...,...
1465433,WA22557C,ELECTRD ELECTROSURGICAL 30D BUTTON HIGH FREQUENCY,NaN,272,NaN,NaN,Max Fee,1585.24
1465434,X-4016,PLEDG CARDIOVASCULAR 10X7MM PTFE PRECUT STERILE,C1768,278,NaN,NaN,Max Fee,54.08
1465435,ZA90030155,LENS INTRAOCULAR +15.5 DIOPTER L13MM DIA6MM RE...,V2632,276,NaN,NaN,Max Fee,527.8
1465436,ZMA00U0185,LENS IOL 5 D +18.5 DIOPTER MODIFY C BICONVEX 1...,V2632,276,NaN,NaN,Max Fee,2606.24


In [11]:
df_mid.loc[df_mid['ms_drg'].notnull(), 'ms_drg'] = df_mid[df_mid['ms_drg'].notnull()]['ms_drg'].str.replace('MS', '')

In [12]:
df_mid.loc[df_mid['rev_code'].notnull(), 'rev_code'] = df_mid[df_mid['rev_code'].notnull()]['rev_code'].str.zfill(4)

In [13]:
df_mid.loc[df_mid['hcpcs_cpt'].notnull(), 'hcpcs_cpt'] = df_mid[df_mid['hcpcs_cpt'].notnull()]['hcpcs_cpt'].str.upper()

In [14]:
def payer_category_from_payer_name(payer_name):
    if payer_name == "Charges":
        return "gross"
    elif payer_name == "Self-pay":
        return "cash"
    elif payer_name == "Min Fee":
        return "min"
    elif payer_name == "Max Fee":
        return "max"

    return "payer"

df_mid['payer_category'] = df_mid['payer_name'].apply(payer_category_from_payer_name)
df_mid

,local_code,description,hcpcs_cpt,rev_code,ms_drg,ndc,payer_name,standard_charge,payer_category
0,18600179,K-WIRE SMOOTH 9 .035,C1713,0278,NaN,NaN,Charges,74.88,gross
1,18600415,WIRE FIXATION 12.7CM .028IN KIRSCHNER STAINLES...,C1713,0278,NaN,NaN,Charges,34.77,gross
2,18602712,WIRE FIXATION 15.2CM .045IN KIRSCHNER STAINLES...,C1713,0278,NaN,NaN,Charges,34.77,gross
3,18602720,WIRE FIXATION 15.2CM .078IN KIRSCHNER STAINLES...,C1713,0278,NaN,NaN,Charges,40.77,gross
4,18604400,WIRE FIXATION TROCAR POINT STYLE BOTH END 15.2...,C1713,0278,NaN,NaN,Charges,47.76,gross
...,...,...,...,...,...,...,...,...,...
1465433,WA22557C,ELECTRD ELECTROSURGICAL 30D BUTTON HIGH FREQUENCY,NaN,0272,NaN,NaN,Max Fee,1585.24,max
1465434,X-4016,PLEDG CARDIOVASCULAR 10X7MM PTFE PRECUT STERILE,C1768,0278,NaN,NaN,Max Fee,54.08,max
1465435,ZA90030155,LENS INTRAOCULAR +15.5 DIOPTER L13MM DIA6MM RE...,V2632,0276,NaN,NaN,Max Fee,527.8,max
1465436,ZMA00U0185,LENS IOL 5 D +18.5 DIOPTER MODIFY C BICONVEX 1...,V2632,0276,NaN,NaN,Max Fee,2606.24,max


In [15]:
df_mid.dropna(subset=['standard_charge'], inplace=True)

In [16]:
df_mid['hospital_id'] = ccn
df_mid['line_type'] = None
df_mid['code'] = None
df_mid['apr_drg'] = None
df_mid['eapg'] = None
df_mid['modifiers'] = None
df_mid['alt_hcpcs_cpt'] = None
df_mid['thru'] = None
df_mid['apc'] = None
df_mid['icd'] = None
df_mid['ndc'] = None
df_mid['drug_hcpcs_multiplier'] = None
df_mid['drug_quantity'] = None
df_mid['drug_unit_of_measurement'] = None
df_mid['drug_type_of_measurement'] = None
df_mid['billing_class'] = None
df_mid['setting'] = None
df_mid['plan_name'] = None
df_mid['standard_charge_percent'] = None
df_mid['contracting_method'] = None
df_mid['additional_generic_notes'] = None
df_mid['additional_payer_specific_notes'] = None

df_mid

,local_code,description,hcpcs_cpt,rev_code,ms_drg,ndc,payer_name,standard_charge,payer_category,hospital_id,...,drug_quantity,drug_unit_of_measurement,drug_type_of_measurement,billing_class,setting,plan_name,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,18600179,K-WIRE SMOOTH 9 .035,C1713,0278,NaN,None,Charges,74.88,gross,110064,...,None,None,None,None,None,None,None,None,None,None
1,18600415,WIRE FIXATION 12.7CM .028IN KIRSCHNER STAINLES...,C1713,0278,NaN,None,Charges,34.77,gross,110064,...,None,None,None,None,None,None,None,None,None,None
2,18602712,WIRE FIXATION 15.2CM .045IN KIRSCHNER STAINLES...,C1713,0278,NaN,None,Charges,34.77,gross,110064,...,None,None,None,None,None,None,None,None,None,None
3,18602720,WIRE FIXATION 15.2CM .078IN KIRSCHNER STAINLES...,C1713,0278,NaN,None,Charges,40.77,gross,110064,...,None,None,None,None,None,None,None,None,None,None
4,18604400,WIRE FIXATION TROCAR POINT STYLE BOTH END 15.2...,C1713,0278,NaN,None,Charges,47.76,gross,110064,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465433,WA22557C,ELECTRD ELECTROSURGICAL 30D BUTTON HIGH FREQUENCY,NaN,0272,NaN,None,Max Fee,1585.24,max,110064,...,None,None,None,None,None,None,None,None,None,None
1465434,X-4016,PLEDG CARDIOVASCULAR 10X7MM PTFE PRECUT STERILE,C1768,0278,NaN,None,Max Fee,54.08,max,110064,...,None,None,None,None,None,None,None,None,None,None
1465435,ZA90030155,LENS INTRAOCULAR +15.5 DIOPTER L13MM DIA6MM RE...,V2632,0276,NaN,None,Max Fee,527.8,max,110064,...,None,None,None,None,None,None,None,None,None,None
1465436,ZMA00U0185,LENS IOL 5 D +18.5 DIOPTER MODIFY C BICONVEX 1...,V2632,0276,NaN,None,Max Fee,2606.24,max,110064,...,None,None,None,None,None,None,None,None,None,None


In [17]:
df_out = pd.DataFrame(df_mid[TARGET_COLUMNS])
df_out

,hospital_id,line_type,description,rev_code,local_code,code,ms_drg,apr_drg,eapg,hcpcs_cpt,...,billing_class,setting,payer_category,payer_name,plan_name,standard_charge,standard_charge_percent,contracting_method,additional_generic_notes,additional_payer_specific_notes
0,110064,None,K-WIRE SMOOTH 9 .035,0278,18600179,None,NaN,None,None,C1713,...,None,None,gross,Charges,None,74.88,None,None,None,None
1,110064,None,WIRE FIXATION 12.7CM .028IN KIRSCHNER STAINLES...,0278,18600415,None,NaN,None,None,C1713,...,None,None,gross,Charges,None,34.77,None,None,None,None
2,110064,None,WIRE FIXATION 15.2CM .045IN KIRSCHNER STAINLES...,0278,18602712,None,NaN,None,None,C1713,...,None,None,gross,Charges,None,34.77,None,None,None,None
3,110064,None,WIRE FIXATION 15.2CM .078IN KIRSCHNER STAINLES...,0278,18602720,None,NaN,None,None,C1713,...,None,None,gross,Charges,None,40.77,None,None,None,None
4,110064,None,WIRE FIXATION TROCAR POINT STYLE BOTH END 15.2...,0278,18604400,None,NaN,None,None,C1713,...,None,None,gross,Charges,None,47.76,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465433,110064,None,ELECTRD ELECTROSURGICAL 30D BUTTON HIGH FREQUENCY,0272,WA22557C,None,NaN,None,None,NaN,...,None,None,max,Max Fee,None,1585.24,None,None,None,None
1465434,110064,None,PLEDG CARDIOVASCULAR 10X7MM PTFE PRECUT STERILE,0278,X-4016,None,NaN,None,None,C1768,...,None,None,max,Max Fee,None,54.08,None,None,None,None
1465435,110064,None,LENS INTRAOCULAR +15.5 DIOPTER L13MM DIA6MM RE...,0276,ZA90030155,None,NaN,None,None,V2632,...,None,None,max,Max Fee,None,527.8,None,None,None,None
1465436,110064,None,LENS IOL 5 D +18.5 DIOPTER MODIFY C BICONVEX 1...,0276,ZMA00U0185,None,NaN,None,None,V2632,...,None,None,max,Max Fee,None,2606.24,None,None,None,None


In [18]:
df_out.to_csv("rate_" + ccn + ".csv", index=False)

In [23]:
from dateutil.parser import parse as parse_datetime

df_tmp = pd.read_excel(filename)
last_updated = df_tmp.iloc[1][0]
last_updated = last_updated.split(' ')[-1]
last_updated = parse_datetime(last_updated).isoformat().split("T")[0]
last_updated

'2022-01-01'